In [1]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import snowflake.connector

from dotenv import load_dotenv
import os
import http.client
import urllib.parse


from googletrans import Translator
import string
import re

from sqlalchemy import create_engine

In [2]:
load_dotenv()
rapidapi_key = os.getenv('RAPIDAPI_KEY')
rapidapi_host = "linkedin-job-search-api.p.rapidapi.com"
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')

In [4]:
def extract_linkedin_job_data():

    headers = {
        'x-rapidapi-key': rapidapi_key,
        'x-rapidapi-host': rapidapi_host
    }

    location = "Australia"
    limit = 100
    offset = 0
    titles = ["Data Engineer" , "Data Scientist", "Data Analyst"]

    df_daily_all = pd.DataFrame()

    for title_filter in titles:

        #URL code the title filter and location filter
        title_encoded=urllib.parse.quote(title_filter)
        location_encoded=urllib.parse.quote(location)

        #API endpoint 7day data query
        #base_url = f"/active-jb-7d?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"

        #API endpoint last 24 hour data query
        base_url = f"/active-jb-24h?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"
        url =  f"https://{rapidapi_host}{base_url}"


        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            # Convert the JSON data to a DataFrame
            df_daily = json_normalize(data)
            df_daily['job_category'] = title_filter
            print(title_filter, df_daily.shape)
            df_daily_all = pd.concat([df_daily_all, df_daily], ignore_index=True)


        else:
            print(f"Error: {response.status_code}")

    return df_daily_all

df_daily_all = extract_linkedin_job_data()
df_daily_all

Data Engineer (20, 51)
Data Scientist (13, 51)
Data Analyst (17, 51)


,id,date_posted,date_created,title,organization,organization_url,date_validthrough,locations_raw,location_type,location_requirements_raw,...,seniority,directapply,linkedin_org_slug,salary_raw.@type,salary_raw.currency,salary_raw.value.@type,salary_raw.value.minValue,salary_raw.value.maxValue,salary_raw.value.unitText,job_category
0,1609222676,2025-05-01T05:22:25,2025-05-01T05:33:40.165725,Data Centre Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-05-31T05:22:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,False,peoplebank,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
1,1609020643,2025-05-01T04:13:16,2025-05-01T04:22:38.862019,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-05-11T04:12:29,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Intermedio,True,talent-international,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
2,1609020610,2025-05-01T04:00:37,2025-05-01T04:22:38.026559,Data Engineer/Big Data Engineer,Renaissance InfoSystems,https://www.linkedin.com/company/renaissance-i...,2025-05-31T04:00:37,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,True,renaissance-infosystems,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
3,1608694859,2025-05-01T02:27:38,2025-05-01T02:33:46.383406,Data Operations Engineer,The Lottery Corporation,https://www.linkedin.com/company/the-lottery-c...,2025-05-31T02:27:03,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Entry level,False,the-lottery-corporation,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
4,1608672851,2025-05-01T02:03:49,2025-05-01T02:23:03.056709,Data Engineer,NRI Australia & New Zealand,https://www.linkedin.com/company/nri-anz,2025-05-31T02:03:48,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Medarbeider,True,nri-anz,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
5,1608615882,2025-05-01T01:50:18,2025-05-01T02:04:03.222583,Data Engineer,NRI Australia & New Zealand,https://www.linkedin.com/company/nri-anz,2025-05-31T01:50:18,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Associate,True,nri-anz,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
6,1608615868,2025-05-01T01:48:17,2025-05-01T02:04:02.789618,Data Engineer,NRI Australia & New Zealand,https://www.linkedin.com/company/nri-anz,2025-05-31T01:48:17,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Associate,True,nri-anz,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
7,1608527284,2025-05-01T01:27:01,2025-05-01T01:35:21.754922,Senior Data Engineer,Hays,https://www.linkedin.com/company/hays,2025-05-15T01:26:20,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,True,hays,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
8,1608320855,2025-05-01T00:06:57,2025-05-01T00:25:28.738569,"Data engineer - DBT, Pyspark",CareCone Group,https://www.linkedin.com/company/carecone,2025-05-31T00:06:56,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,True,carecone,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
9,1608266328,2025-04-30T23:56:49,2025-05-01T00:03:48.144027,Senior Data Engineer,Reejig,https://www.linkedin.com/company/reejig,2025-10-27T23:56:38,"[{'@type': 'Place', 'address': {'@type': 'Post...",TELECOMMUTE,"[{'@type': 'Country', 'name': 'Sydney, New Sou...",...,Mid-Senior level,True,reejig,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer


In [5]:
df_daily_all.to_csv('linkedin_jobs_daily.csv', index=False)

In [6]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.shape

(50, 51)

In [7]:
def update_columns(df_daily_all):
    df_daily_all.columns = df_daily_all.columns.str.upper()
    df_daily_all = df_daily_all[['ID', 'DATE_POSTED', 'DATE_CREATED', 'TITLE', 'JOB_CATEGORY',
       'ORGANIZATION', 'ORGANIZATION_URL', 'DATE_VALIDTHROUGH', 'LOCATIONS_RAW',
       'LOCATION_TYPE', 'LOCATION_REQUIREMENTS_RAW', 'EMPLOYMENT_TYPE', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN', 'ORGANIZATION_LOGO',
       'CITIES_DERIVED', 'REGIONS_DERIVED', 'COUNTRIES_DERIVED',
       'LOCATIONS_DERIVED', 'TIMEZONES_DERIVED', 'LATS_DERIVED',
       'LNGS_DERIVED', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_EMPLOYEES', 'LINKEDIN_ORG_URL',
       'LINKEDIN_ORG_SIZE', 'LINKEDIN_ORG_SLOGAN', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_FOLLOWERS', 'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION', 'LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY', 'LINKEDIN_ORG_SLUG']]
    return df_daily_all

df_daily_all = update_columns(df_daily_all)
df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1609222676,2025-05-01T05:22:25,2025-05-01T05:33:40.165725,Data Centre Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-05-31T05:22:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank
1,1609020643,2025-05-01T04:13:16,2025-05-01T04:22:38.862019,Data Engineer,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-05-11T04:12:29,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1995.0,"['IT and Technology Recruitment Specialist', '...","['Level 12, 2 Park Street, Sydney, NSW 2000, A...",Talent is a global recruitment specialist whic...,True,Intermedio,True,talent-international
2,1609020610,2025-05-01T04:00:37,2025-05-01T04:22:38.026559,Data Engineer/Big Data Engineer,Data Engineer,Renaissance InfoSystems,https://www.linkedin.com/company/renaissance-i...,2025-05-31T04:00:37,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Melbourne, Victoria",Privately Held,2006.0,[''],"['Suite 508/Level 5, 343 Little Collins Street...",Renaissance InfoSystems is a technology and di...,True,Mid-Senior level,True,renaissance-infosystems
3,1608694859,2025-05-01T02:27:38,2025-05-01T02:33:46.383406,Data Operations Engineer,Data Engineer,The Lottery Corporation,https://www.linkedin.com/company/the-lottery-c...,2025-05-31T02:27:03,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Brisbane, Queensland",Public Company,2022.0,['Lotteries'],"['180 Ann St, Brisbane, Queensland 4000, AU']",We’re Australia’s largest lottery company offe...,False,Entry level,False,the-lottery-corporation
4,1608672851,2025-05-01T02:03:49,2025-05-01T02:23:03.056709,Data Engineer,Data Engineer,NRI Australia & New Zealand,https://www.linkedin.com/company/nri-anz,2025-05-31T02:03:48,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, Australia, New South Wales",Privately Held,1996.0,"['IT Consulting', 'Managed Services', 'Cloud',...","['171 Sussex St, Level 21, Sydney, Australia, ...",Transformative digital solutions take more tha...,False,Medarbeider,True,nri-anz


## Connect to  Snowflake database for raw data initial processing

In [4]:
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()


Connection to Snowflake established successfully.


In [9]:
#query the raw data
def query_raw_api_data(conn):
    query = """
        SELECT * FROM clean_job_title_data
    """

    df = pd.read_sql(query, conn)
    print(df.shape)
    return df

df = query_raw_api_data(conn)
df.head()

/tmp/ipykernel_5254/2965248746.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(182, 31)


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1599037955,Senior Data Engineer,Data Engineer,2025-04-28,Greater Sydney Area,None,FULL_TIME,One51 | Data & Analytics Consultancy,https://www.linkedin.com/company/one51consulting,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2020.0,"['Business Intelligence', 'Data Warehousing', ...","['333 George Street, Level 13, Sydney, NSW 200...",Drawing on a wealth of expertise and a deep un...,False,Mid-Senior level,False,one51consulting
1,1598878880,Senior Data Engineer,Data Engineer,2025-04-28,Brisbane,QLD,CONTRACTOR,Data#3,https://www.linkedin.com/company/data3,https://au.linkedin.com/jobs/view/senior-data-...,...,"Toowong, Queensland",Public Company,1977.0,"['Cloud Solutions', 'Mobility Solutions', 'Sec...","['555 Coronation Dr, Toowong, Queensland 4066,...","Data#3 Limited (DTL), is focused on helping cu...",False,Livello medio-alto,False,data3
2,1598880069,Data Engineer,Data Engineer,2025-04-28,Sydney,NSW,CONTRACTOR,Whizdom,https://www.linkedin.com/company/whizdom-recru...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Canberra, Australian Capital Territory",Privately Held,2006.0,"['IT Recruitment', 'Recruitment for Government...","['28-34 Thynne St, Unit 7, Canberra, Australia...","Established in 2006, Whizdom is an Australian ...",True,Mellannivå,True,whizdom-recruitment
3,1598878894,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Moreton Bay,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None
4,1598880083,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Sunshine Coast,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None


### Process the df_daily_all data 


In [10]:
def extract_job_date(date_created):
    #extract the job_date from job_created
    df_daily_all['job_date'] = pd.to_datetime(df_daily_all['DATE_CREATED']).dt.date
    return df_daily_all['job_date']

#Apply the function to the DataFrame
df_daily_all['job_date'] = extract_job_date(df_daily_all['DATE_CREATED'])
df_daily_all['job_date'][:5]


0    2025-05-01
1    2025-05-01
2    2025-05-01
3    2025-05-01
4    2025-05-01
Name: job_date, dtype: object

In [11]:
#Extrac the job city and state from the LOCATIONS_RAW field

def extract_city(list):
    """
    Extracts the city from the given text using regex.
    """
    # Regex pattern to match the city in the LOCATIONS_RAW field
    text = str(list)
    city_pattern = r"'addressLocality':\s*'(.*)',\s'addressRegion':"
    match = re.search(city_pattern, text)
    if match:
        city = match.group(1)
        if 'sidney' in city.lower() or 'sídney' in city.lower() or '悉尼' in city.lower(): #fix wrong city name
            return "Sydney"
        return city
    else:
        return None



def extract_state(list):
    """
    Extracts the state from the given text using regex.
    """
    # Regex pattern to match the region/state in the LOCATIONS_RAW field
    text = str(list)
    state_pattern = r"'addressRegion':\s*(.*)',\s'streetAddress'"
    match = re.search(state_pattern, text)
    if match:
        state = match.group(1)
        state= state.replace("'", "").strip()
        return state
    else:
        return None


#Extract city and state from Locations Raw
df_daily_all['city'] = df_daily_all['LOCATIONS_RAW'].apply(extract_city)


df_daily_all['state'] = df_daily_all['LOCATIONS_RAW'].apply(extract_state)



df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,job_date,city,state
0,1609222676,2025-05-01T05:22:25,2025-05-01T05:33:40.165725,Data Centre Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-05-31T05:22:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank,2025-05-01,Melbourne,VIC
1,1609020643,2025-05-01T04:13:16,2025-05-01T04:22:38.862019,Data Engineer,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-05-11T04:12:29,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['IT and Technology Recruitment Specialist', '...","['Level 12, 2 Park Street, Sydney, NSW 2000, A...",Talent is a global recruitment specialist whic...,True,Intermedio,True,talent-international,2025-05-01,capital Australia,None
2,1609020610,2025-05-01T04:00:37,2025-05-01T04:22:38.026559,Data Engineer/Big Data Engineer,Data Engineer,Renaissance InfoSystems,https://www.linkedin.com/company/renaissance-i...,2025-05-31T04:00:37,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,[''],"['Suite 508/Level 5, 343 Little Collins Street...",Renaissance InfoSystems is a technology and di...,True,Mid-Senior level,True,renaissance-infosystems,2025-05-01,Sydney,NSW
3,1608694859,2025-05-01T02:27:38,2025-05-01T02:33:46.383406,Data Operations Engineer,Data Engineer,The Lottery Corporation,https://www.linkedin.com/company/the-lottery-c...,2025-05-31T02:27:03,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,['Lotteries'],"['180 Ann St, Brisbane, Queensland 4000, AU']",We’re Australia’s largest lottery company offe...,False,Entry level,False,the-lottery-corporation,2025-05-01,Melbourne,VIC
4,1608672851,2025-05-01T02:03:49,2025-05-01T02:23:03.056709,Data Engineer,Data Engineer,NRI Australia & New Zealand,https://www.linkedin.com/company/nri-anz,2025-05-31T02:03:48,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['IT Consulting', 'Managed Services', 'Cloud',...","['171 Sussex St, Level 21, Sydney, Australia, ...",Transformative digital solutions take more tha...,False,Medarbeider,True,nri-anz,2025-05-01,Perth,WA


In [12]:
#Extract the employment type from the EMPLOYMENT_TYPE field

def extract_employment_type(df_daily_all):
    """
    Extracts the employment type from the given text using regex.
    """
    df_daily_all['employment_type'] = (
        df_daily_all['EMPLOYMENT_TYPE']
        .astype(str)
        .str.replace(r"[\[\]']", '', regex=True)
        .str.strip()
    )

    df_daily_all.drop(columns=['EMPLOYMENT_TYPE'], inplace=True)

    return df_daily_all['employment_type']


df_daily_all['employment_type'] = extract_employment_type(df_daily_all)
df_daily_all['employment_type'].tail()

45    CONTRACTOR
46     FULL_TIME
47     FULL_TIME
48     FULL_TIME
49     FULL_TIME
Name: employment_type, dtype: object

In [13]:
#Extract the employment size
def extract_employee_size(LINKEDIN_ORG_SIZE):
    """
    Extracts the employee size from the given text using regex.
    """
    df_daily_all['org_size'] = (
        df_daily_all['LINKEDIN_ORG_SIZE']
        .astype(str)
        .str.replace(r"employees", '', regex=True)
        .str.strip()
    )

    return df_daily_all['org_size']


df_daily_all['org_size'] = extract_employee_size(df_daily_all['LINKEDIN_ORG_SIZE'])
df_daily_all['org_size'].tail()

45       501-1,000
46             nan
47    5,001-10,000
48         201-500
49    5,001-10,000
Name: org_size, dtype: object

In [14]:
df_daily_all.columns = df_daily_all.columns.str.upper()
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORG_SIZE
0,1609222676,2025-05-01T05:22:25,2025-05-01T05:33:40.165725,Data Centre Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-05-31T05:22:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank,2025-05-01,Melbourne,VIC,CONTRACTOR,201-500
1,1609020643,2025-05-01T04:13:16,2025-05-01T04:22:38.862019,Data Engineer,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-05-11T04:12:29,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,Talent is a global recruitment specialist whic...,True,Intermedio,True,talent-international,2025-05-01,capital Australia,None,CONTRACTOR,201-500


In [15]:
#Only keep the relevant columns

df_daily_all = df_daily_all[['ID', 'TITLE', 'JOB_CATEGORY',
       'JOB_DATE', 'CITY', 'STATE', 'EMPLOYMENT_TYPE' ,
       'ORGANIZATION', 'ORGANIZATION_URL', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN',
       'ORGANIZATION_LOGO', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_URL',
       'ORG_SIZE', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION','LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY',
       'LINKEDIN_ORG_SLUG']]


df_daily_all.head()

,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1609222676,Data Centre Engineer,Data Engineer,2025-05-01,Melbourne,VIC,CONTRACTOR,Peoplebank,https://www.linkedin.com/company/peoplebank,https://au.linkedin.com/jobs/view/data-centre-...,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank
1,1609020643,Data Engineer,Data Engineer,2025-05-01,capital Australia,None,CONTRACTOR,Talent,https://www.linkedin.com/company/talent-intern...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Sydney, NSW",Privately Held,1995.0,"['IT and Technology Recruitment Specialist', '...","['Level 12, 2 Park Street, Sydney, NSW 2000, A...",Talent is a global recruitment specialist whic...,True,Intermedio,True,talent-international
2,1609020610,Data Engineer/Big Data Engineer,Data Engineer,2025-05-01,Sydney,NSW,FULL_TIME,Renaissance InfoSystems,https://www.linkedin.com/company/renaissance-i...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Melbourne, Victoria",Privately Held,2006.0,[''],"['Suite 508/Level 5, 343 Little Collins Street...",Renaissance InfoSystems is a technology and di...,True,Mid-Senior level,True,renaissance-infosystems
3,1608694859,Data Operations Engineer,Data Engineer,2025-05-01,Melbourne,VIC,FULL_TIME,The Lottery Corporation,https://www.linkedin.com/company/the-lottery-c...,https://au.linkedin.com/jobs/view/data-operati...,...,"Brisbane, Queensland",Public Company,2022.0,['Lotteries'],"['180 Ann St, Brisbane, Queensland 4000, AU']",We’re Australia’s largest lottery company offe...,False,Entry level,False,the-lottery-corporation
4,1608672851,Data Engineer,Data Engineer,2025-05-01,Perth,WA,FULL_TIME,NRI Australia & New Zealand,https://www.linkedin.com/company/nri-anz,https://au.linkedin.com/jobs/view/data-enginee...,...,"Sydney, Australia, New South Wales",Privately Held,1996.0,"['IT Consulting', 'Managed Services', 'Cloud',...","['171 Sussex St, Level 21, Sydney, Australia, ...",Transformative digital solutions take more tha...,False,Medarbeider,True,nri-anz


In [16]:
#Merge the new data with the existing data

def merge_duplicates(df, df_daily_all):
    # Merge the two DataFrames
    merged_df = pd.concat([df, df_daily_all], axis=0)

    # Drop duplicates based on 'job_id' and keep the last occurrence
    merged_df.drop_duplicates(subset=['ID'], keep='last', inplace=True)

    # Reset the index
    merged_df.reset_index(drop=True, inplace=True)

    return merged_df

df_merged = merge_duplicates(df, df_daily_all)
df_merged.shape


(232, 31)

In [17]:
df_merged.tail()

,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
227,1605434899,Business Data Analyst,Data Analyst,2025-04-30,Sydney,NSW,CONTRACTOR,N2S.Global,https://www.linkedin.com/company/n2sglobal,https://au.linkedin.com/jobs/view/business-dat...,...,"Plano, TX",Privately Held,NaN,"['Payroll Solutions', 'Compliance Solutions', ...","['2805 North Dallas Parkway, Suite 110, Plano,...",Welcome to N2S.Global! 🌍\n\nYour partner for s...,False,Algo de responsabilidad,True,n2sglobal
228,1605434919,Data Analyst,Data Analyst,2025-04-30,Canberra,None,FULL_TIME,Australian Automobile Association,https://www.linkedin.com/company/australian-au...,https://au.linkedin.com/jobs/view/data-analyst...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stagiair,True,NaN
229,1605434796,Lead Data Analyst,Data Analyst,2025-04-30,Adelaide,SA,FULL_TIME,Hays,https://www.linkedin.com/company/hays,https://au.linkedin.com/jobs/view/lead-data-an...,...,"London, England",Public Company,NaN,"['Recruitment', 'Staffing solutions', 'Resourc...","['20 Triton Street, London, England NW1 3BF, G...",We are leaders in specialist recruitment and w...,True,Senior medewerker,True,hays
230,1605443097,Electronic Data Interchange Business Analyst,Data Analyst,2025-04-30,Sydney,NSW,FULL_TIME,B2BE,https://www.linkedin.com/company/b2be,https://au.linkedin.com/jobs/view/electronic-d...,...,"Birmingham, West Midlands",Privately Held,1998.0,"['EDI', 'Business to Business eCommerce soluti...","['62 Hagley Road, Level 10, Birmingham, West M...",B2BE’s e-Commerce based products and solutions...,False,Mid-Senior level,True,b2be
231,1605439930,Senior Data Analyst - Procurement,Data Analyst,2025-04-30,Greater Sydney Area,None,FULL_TIME,HealthShare NSW,https://www.linkedin.com/company/healthsharensw,https://au.linkedin.com/jobs/view/senior-data-...,...,"St Leonards, New South Wales",Government Agency,NaN,"['Food and Hotel Services', 'Non-Emergency Pat...","['1 Reserve Rd, St Leonards, New South Wales 2...",HealthShare NSW supports NSW Health to deliver...,False,Associate,False,healthsharensw


In [18]:
#Translate business name, city, state and seniority to English

translator = Translator()
def translate_text(text, target_language='en'):
    try:
        if not text:
            return 'NA'
        translated_text = translator.translate(text, dest=target_language)
        return translated_text.text
    except Exception as e:
        print(f"Error translating text: {e}")
        return text

df_merged['CITY'] = df_merged['CITY'].apply(lambda x: translate_text(x, target_language='en'))

df_merged['STATE'] = df_merged['STATE'].apply(lambda x: translate_text(x, target_language='en'))

df_merged['ORGANIZATION'] = df_merged['ORGANIZATION'].apply(lambda x: translate_text(x, target_language='en'))

df_merged['SENIORITY'] = df_merged['SENIORITY'].apply(lambda x: translate_text(x, target_language='en'))

Error translating text: list index out of range


In [22]:

df_merged['CITY'].nunique()

40

In [ ]:
df_merged['STATE'].unique()

array(['NA', 'QLD', 'NSW', 'VIC', 'NT', 'Australia', 'Of', 'on'],
      dtype=object)

In [ ]:
# #Load processed data into Snowflake

# def load_to_snowflake(df_merged):
#     # Create a Snowflake connection engine
#    engine = create_engine(
#         'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'.format(
#         user="NIKKILW2025",
#         password=snowflake_password,
#         account="gbszkwp-by30611",
#         warehouse="SNOWFLAKE_LEARNING_WH",
#         database="linkedin_db",
#         schema="linkedin_raw"
#     )
#    )

#    table_name = "linkedin_job_api_cleaned_data"

#    df_merged.to_sql(
#         name=table_name,
#         con=engine,
#         if_exists='replace',
#         index=False
#     )

#    print(f"Data loaded to Snowflake table {table_name} successfully.")


# load_to_snowflake(df_merged)

Data loaded to Snowflake table linkedin_job_api_cleaned_data successfully.
